In [ ]:
This model seems to be worst than the second submission

- LB
    - Submssion 2 (Pure Cat) : 0.78435
    - Sumssion 3 (This stacker) : 0.78392

Code Source : [https://www.kaggle.com/hhstrand/oof-stacking-regime]

- First stacking (30th Jan 2019)
- It uses two best scoring kernels so far
    - LGBM (0.7726) : lgbm_baseline_with_ordered_ordinal_0123_1352_0.7726
    - CatBoost (0.7854) : cat_baseline_with_ordered_ordinal_use_cat_feature_0123_1455_0.7854

- In the notebok (stacking_baseline_with_lgbm_cat.ipynb), I have tried
two types of models, LGBM and XGB as the predictors of the second round. Based on the CV score
finally I have selected the XGB.

- Details
    - Level 1 Predictors (LGBM, CatBoost)
    - Level 2 Predictor (XGB)

In [2]:
import sys

import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb

sys.path.insert(0, "/home/jupyter/kaggle/cat_in_dat_2_git/cat_in_dat_2/src")
import utility

## Get the Data

In [3]:
SUBMISSION_DIR = '../../sub'
OOF_DIR = '../../oof'

# Load the original train/test data
dir_path = '/home/jupyter/kaggle/cat_in_dat_2_git/cat_in_dat_2/data/read_only'
SEED = 42
train_file_name = 'train.csv'
test_file_name = 'test.csv'
submission_file_name = 'sample_submission.csv'
index_col = 'id'

train = pd.read_csv(f'{dir_path}/{train_file_name}', index_col=index_col)
test = pd.read_csv(f'{dir_path}/{test_file_name}', index_col=index_col)
submission = pd.read_csv(f'{dir_path}/{submission_file_name}')

print(train.shape, test.shape, submission.shape)

# Load the OOF data
oof_lgbm = pd.read_csv(f'{OOF_DIR}/oof_lgbm_baseline_with_ordered_ordinal_0123_1352_0.7726.csv')
oof_cat = pd.read_csv(f'{OOF_DIR}/oof_cat_baseline_with_ordered_ordinal_use_cat_feature_0123_1455_0.7854.csv')

# Load the prediction data
prediction_lgbm = pd.read_csv(f'{SUBMISSION_DIR}/sub_lgbm_baseline_with_ordered_ordinal_0123_1352_0.7726.csv')
prediction_cat = pd.read_csv(f'{SUBMISSION_DIR}/sub_cat_baseline_with_ordered_ordinal_use_cat_feature_0123_1455_0.7854.csv')

print(f'Shape fo the OOF data {oof_lgbm.shape}, {oof_cat.shape}') 
print(f'Shape fo the Prediction data {prediction_lgbm.shape}, {prediction_cat.shape}')

# Create oof DF
oof_df = pd.concat([oof_lgbm, oof_cat], axis=1)
oof_df.columns = ['lgbm', 'cat']

# Create prediction DF
prediction_df = pd.merge(prediction_lgbm, prediction_cat, 
                         how='left', 
                         on='id')
prediction_df.columns = ['id', 'lgbm', 'cat']

print(f'Shape fo the OOF DF {oof_df.shape}') 
print(f'Shape fo the Prediction DF {prediction_df.shape}')

(600000, 24) (400000, 23) (400000, 2)
Shape fo the OOF data (600000, 1), (600000, 1)
Shape fo the Prediction data (400000, 2), (400000, 2)
Shape fo the OOF DF (600000, 2)
Shape fo the Prediction DF (400000, 3)


## Stack with LGBM as the Second Layer Predictor (Stratified = True)

In [4]:
# Parameters related to model
METRIC = 'auc'
N_ESTIMATORS = 1000
EARLY_STOPPING_ROUNDS = 25
VERBOSE = -1
N_THREADS = -1

# Name of the target
TARGET = 'target'

# Params 
lgb_params = {
    'objective' : 'binary',
    'boosting_type' : 'gbdt',
    'metric' : METRIC,
    'num_threads': N_THREADS,
    'verbose' : VERBOSE,
    'seed': SEED,
    'n_estimators' : N_ESTIMATORS,
    'early_stopping_rounds' : EARLY_STOPPING_ROUNDS
    }

lgb_train = lgb.Dataset(oof_df, train[TARGET])
lgb_test = lgb.Dataset(prediction_df)

lgb_result = lgb.cv(params=lgb_params, train_set=lgb_train, nfold=5, stratified=True, show_stdv=True, verbose_eval=1, shuffle=True)

# The length of this array tells 
# us the number of iterations
# Needed to reach the score.
lgb_best_nrounds = len(lgb_result['auc-mean'])
lgb_cv_mean = np.mean(lgb_result['auc-mean'])
lgb_cv_std = np.mean(lgb_result['auc-stdv'])

print(f'best number of iterations {lgb_best_nrounds}')
print(f'best number of iterations {lgb_cv_mean}')
print(f'best number of iterations {lgb_cv_std}')

/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/lightgbm/engine.py:502: UserWarning:

Found `n_estimators` in params. Will use it instead of argument

/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/lightgbm/engine.py:507: UserWarning:

Found `early_stopping_rounds` in params. Will use it instead of argument



[1]	cv_agg's auc: 0.784748 + 0.00198658
[2]	cv_agg's auc: 0.784962 + 0.00199664
[3]	cv_agg's auc: 0.785009 + 0.00199854
[4]	cv_agg's auc: 0.785119 + 0.00196179
[5]	cv_agg's auc: 0.785165 + 0.00194602
[6]	cv_agg's auc: 0.78517 + 0.00193698
[7]	cv_agg's auc: 0.78518 + 0.00192665
[8]	cv_agg's auc: 0.785194 + 0.00191697
[9]	cv_agg's auc: 0.785198 + 0.00190946
[10]	cv_agg's auc: 0.785206 + 0.00190386
[11]	cv_agg's auc: 0.785212 + 0.00189481
[12]	cv_agg's auc: 0.785229 + 0.00189694
[13]	cv_agg's auc: 0.785243 + 0.00190964
[14]	cv_agg's auc: 0.785253 + 0.00190446
[15]	cv_agg's auc: 0.785244 + 0.00191067
[16]	cv_agg's auc: 0.785238 + 0.00190621
[17]	cv_agg's auc: 0.785244 + 0.00190572
[18]	cv_agg's auc: 0.785251 + 0.00190257
[19]	cv_agg's auc: 0.78526 + 0.00190216
[20]	cv_agg's auc: 0.785262 + 0.00190696
[21]	cv_agg's auc: 0.785263 + 0.00190361
[22]	cv_agg's auc: 0.785262 + 0.00189499
[23]	cv_agg's auc: 0.785269 + 0.00189881
[24]	cv_agg's auc: 0.785265 + 0.00189545
[25]	cv_agg's auc: 0.785257 

## Stack with LGBM as the Second Layer Predictor (Stratified = False)

In [5]:
lgb_result = lgb.cv(params=lgb_params, train_set=lgb_train, nfold=5, stratified=True, show_stdv=True, verbose_eval=1, shuffle=False)

# The length of this array tells 
# us the number of iterations
# Needed to reach the score.
lgb_best_nrounds = len(lgb_result['auc-mean'])
lgb_cv_mean = np.mean(lgb_result['auc-mean'])
lgb_cv_std = np.mean(lgb_result['auc-stdv'])

print(f'best number of iterations {lgb_best_nrounds}')
print(f'best number of iterations {lgb_cv_mean}')
print(f'best number of iterations {lgb_cv_std}')

/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/lightgbm/engine.py:502: UserWarning:

Found `n_estimators` in params. Will use it instead of argument

/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/lightgbm/engine.py:507: UserWarning:

Found `early_stopping_rounds` in params. Will use it instead of argument



[1]	cv_agg's auc: 0.784719 + 0.000586302
[2]	cv_agg's auc: 0.784972 + 0.000607408
[3]	cv_agg's auc: 0.785074 + 0.000619804
[4]	cv_agg's auc: 0.785124 + 0.00063039
[5]	cv_agg's auc: 0.785152 + 0.000637363
[6]	cv_agg's auc: 0.785189 + 0.000641033
[7]	cv_agg's auc: 0.785222 + 0.000639568
[8]	cv_agg's auc: 0.785224 + 0.000641326
[9]	cv_agg's auc: 0.785239 + 0.000657198
[10]	cv_agg's auc: 0.785256 + 0.000641546
[11]	cv_agg's auc: 0.785257 + 0.000644141
[12]	cv_agg's auc: 0.78526 + 0.00064082
[13]	cv_agg's auc: 0.785266 + 0.000640836
[14]	cv_agg's auc: 0.785261 + 0.000643215
[15]	cv_agg's auc: 0.785266 + 0.000648075
[16]	cv_agg's auc: 0.785263 + 0.000648925
[17]	cv_agg's auc: 0.785269 + 0.000647956
[18]	cv_agg's auc: 0.785276 + 0.000648615
[19]	cv_agg's auc: 0.785273 + 0.000657286
[20]	cv_agg's auc: 0.78527 + 0.00065581
[21]	cv_agg's auc: 0.785271 + 0.000660348
[22]	cv_agg's auc: 0.785275 + 0.000658857
[23]	cv_agg's auc: 0.785272 + 0.000654387
[24]	cv_agg's auc: 0.785275 + 0.000649109
[25]	c

## Stack with XGB as the Second Layer Predictor (Stratified = True)

In [18]:
# Parameters related to KFold
N_FOLDS = 10
SHUFFLE = True

# Parameters related to model
MODEL_TYPE = "xgb"
METRIC = 'auc'
N_ESTIMATORS = 1000
EARLY_STOPPING_ROUNDS = 25
VERBOSE = 1
N_THREADS = -1

# Name of the target
TARGET = 'target'

xgb_params = {
    'objective' : 'binary:logistic',
    'booster' : 'gbtree',
    'eval_metric' : METRIC,
    'nthread': N_THREADS,
    'seed': SEED,
    'validate_parameters' : True
    }

xgb_train = xgb.DMatrix(data=oof_df, label=train[TARGET])
xgb_test = xgb.DMatrix(prediction_df[['lgbm', 'cat']])

results_xgb = xgb.cv(params=xgb_params, 
                     dtrain=xgb_train, 
                     num_boost_round=N_ESTIMATORS, 
                     stratified=True,
                     nfold=5, 
                     early_stopping_rounds=25,
                     show_stdv=True, 
                     shuffle=True, 
                     verbose_eval=1)

xgb_best_nrounds = results_xgb.shape[0]
xgb_cv_mean = results_xgb.iloc[-1, 0]
xgb_cv_std = results_xgb.iloc[-1, 1]

print(f'best number of iterations {xgb_best_nrounds}')
print(f'best number of iterations {xgb_cv_mean}')
print(f'best number of iterations {xgb_cv_std}')

/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



[0]	train-auc:0.785585+0.000476815	test-auc:0.784719+0.00185479
[1]	train-auc:0.785993+0.000471656	test-auc:0.784932+0.00189893
[2]	train-auc:0.786226+0.000485657	test-auc:0.785041+0.00192133
[3]	train-auc:0.786381+0.000472762	test-auc:0.785074+0.00193066
[4]	train-auc:0.786544+0.000467533	test-auc:0.785056+0.00194704
[5]	train-auc:0.786673+0.000473325	test-auc:0.785059+0.00194739
[6]	train-auc:0.786797+0.000477298	test-auc:0.785091+0.00192539
[7]	train-auc:0.786929+0.00048862	test-auc:0.785084+0.00193133
[8]	train-auc:0.787053+0.000489087	test-auc:0.785071+0.0019432
[9]	train-auc:0.787155+0.000491343	test-auc:0.785055+0.00194661
[10]	train-auc:0.787261+0.000484314	test-auc:0.785068+0.00194266
[11]	train-auc:0.787368+0.000502516	test-auc:0.785067+0.00196304
[12]	train-auc:0.787475+0.000506328	test-auc:0.785047+0.0019676
[13]	train-auc:0.787567+0.000500398	test-auc:0.785037+0.00194859
[14]	train-auc:0.787655+0.00048959	test-auc:0.785029+0.00193679
[15]	train-auc:0.78776+0.000514095	test

#### Explanation of the output above
Here, I am printing at the end of each iteration.
- Each line above represents the train and test auc score averaged over nfolds (5 here) and corresponding std.
- The best test-auc-mean is reached at the 6th (0 index based) iteration.
- Since early stopping round is set to 25, XGB continued to train for another 25 iteration (including 6th) and then stopped.
- results_xgb represents the train and test auc score averaged over nfolds till the 6th iteration (0 index based)

In [12]:
# Print XGB
results_xgb

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.785628,0.000299,0.784596,0.000924
1,0.786049,0.000268,0.784859,0.000980
2,0.786252,0.000262,0.784962,0.001026
3,0.786412,0.000270,0.784988,0.001013
4,0.786559,0.000270,0.784985,0.001024
5,0.786673,0.000245,0.784989,0.001025
6,0.786819,0.000269,0.785000,0.001045
7,0.786946,0.000280,0.785026,0.001045
8,0.787069,0.000283,0.785036,0.001025


## Stack with XGB as the Second Layer Predictor (Stratified = False)

In [14]:
results_xgb = xgb.cv(params=xgb_params, 
                     dtrain=xgb_train, 
                     num_boost_round=N_ESTIMATORS, 
                     stratified=False,
                     nfold=5, 
                     early_stopping_rounds=25,
                     show_stdv=True, 
                     shuffle=True, 
                     verbose_eval=1)

xgb_best_nrounds = results_xgb.shape[0]
xgb_cv_mean = results_xgb.iloc[-1, 0]
xgb_cv_std = results_xgb.iloc[-1, 1]

print(f'best number of iterations {xgb_best_nrounds}')
print(f'best number of iterations {xgb_cv_mean}')
print(f'best number of iterations {xgb_cv_std}')

[0]	train-auc:0.785628+0.000298992	test-auc:0.784596+0.000923552
[1]	train-auc:0.786049+0.000268235	test-auc:0.784859+0.000979865
[2]	train-auc:0.786252+0.000262263	test-auc:0.784962+0.00102623
[3]	train-auc:0.786412+0.000270374	test-auc:0.784988+0.00101297
[4]	train-auc:0.786559+0.000269792	test-auc:0.784985+0.001024
[5]	train-auc:0.786673+0.00024544	test-auc:0.784989+0.00102456
[6]	train-auc:0.786819+0.000269318	test-auc:0.785+0.00104529
[7]	train-auc:0.786946+0.000280137	test-auc:0.785026+0.00104471
[8]	train-auc:0.787069+0.000283464	test-auc:0.785036+0.00102464
[9]	train-auc:0.787184+0.000289438	test-auc:0.785016+0.00104636
[10]	train-auc:0.7873+0.00028956	test-auc:0.785022+0.00107733
[11]	train-auc:0.7874+0.000281698	test-auc:0.784993+0.00106041
[12]	train-auc:0.787513+0.000266138	test-auc:0.784997+0.00104246
[13]	train-auc:0.787623+0.000268869	test-auc:0.78499+0.00103779
[14]	train-auc:0.787707+0.000281389	test-auc:0.784983+0.00103927
[15]	train-auc:0.787797+0.00027521	test-auc:0

#### Since, the XGB with Startified False has a better CV score, I will go ahead with XGB.

- XGB (Startified = True) :  0.7867966 / 0.00047729848103676925
- XGB (Startified = False) : 0.787069 / 0.0002834635779072801

Note : While doing the final model building, Startified True or False doesn't make any sense.
Stratification is useful for CV only.

- I will use the number of best iterations based on the CV.

## Predict

In [21]:
# Build the final model for predcition
# OOF predictions from level 1 are the independent variables.
# Number of iterations are selected from the cross validation above
xgb_final = xgb.train(xgb_params, dtrain=xgb_train, num_boost_round=xgb_best_nrounds)

In [22]:
# Predict
# Independent variables are the predictions from Level 1
# That is the submissions from level 1
prediction_test = xgb_final.predict(xgb_test)

print(prediction_test.shape)

submission.target = prediction_test

submission.head()

(400000,)


In [26]:
# Build the submission name
submission_file_name = '3_sub_xgb_stacker_baseline_0130_1057_0.7871.csv'

submission.to_csv(f'{SUBMISSION_DIR}/{submission_file_name}', index=False)

In [31]:
f'{SUBMISSION_DIR}/{submission_file_name}'

'../../sub/3_sub_xgb_stacker_baseline_0130_1057_0.7871.csv'

In [32]:
! /home/jupyter/.local/bin/kaggle competitions submit -c cat-in-the-dat-ii -f ../../sub/3_sub_xgb_stacker_baseline_0130_1057_0.7871.csv -m "Baseline stacking with LGBM & Cat base models"

100%|██████████| 6.86M/6.86M [00:00<00:00, 17.4MB/s]
Successfully submitted to Categorical Feature Encoding Challenge II